# Boston housing prices linear regression

In [ ]:
import pandas as pd
import numpy as np
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor as VIF
from ISLP import load_data

import matplotlib.pyplot as plt
import seaborn as sns

## Prepare the data

In [ ]:
Boston = load_data("Boston")
Boston.columns

* **crim**: per capita crime rate by town.
* **zn**: proportion of residential land zoned for lots over 25,000 sq.ft.
* **indus**: proportion of non-retail business acres per town.
* **chas**: Charles River dummy variable (= 1 if tract bounds river; 0 otherwise).
* **nox**: nitrogen oxides concentration (parts per 10 million).
* **rm**: average number of rooms per dwelling.
* **age**: proportion of owner-occupied units built prior to 1940.
* **dis**: weighted mean of distances to five Boston employment centres.
* **rad**: index of accessibility to radial highways.
* **tax**: full-value property-tax rate per \$10,000.
* **ptratio**: pupil-teacher ratio by town.
* **lstat**: lower status of the population (percent).
* **medv**: median value of owner-occupied homes in \$1000s.

In [ ]:
target = 'medv'
features = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'lstat']

In [ ]:
y = Boston[target]
X = Boston[features].copy()
X['intercept'] = 1.0

`statsmodels` requires us to add intercept manually

## Fit the model

In [ ]:
model = sm.OLS(y, X)
results = model.fit()

Most of the subsequent work will be done with the `results` - a result wrapper returned by the fit model.

It contains:
 - `summary()`
 - `get_prediction()`
 - `fittedvalues`
 - `resid`
 - `get_influence()`

In [ ]:
results.summary()

#### Observations

1. `statsmodels` tells that we might have a multicollinearity problem (condition number is big).
2. Two featrues are insignificant: `age` and `indus`.

## Predictions

In [ ]:
y_test = y.iloc[-2:]
X_test = X.iloc[-2:]

In [ ]:
prediction = results.get_prediction(X_test)

`get_prediction()` returns a result wrapper that contains:
- `predicted_mean`
- `conf_int(obs=False)` - confidence interval
- `conf_int(obs=True)` - prediction interval

In [ ]:
print("expected: ", y_test.values)
print("predicted: ", prediction.predicted_mean.round(1))

#### Confidence Interval

In [ ]:
prediction.conf_int(obs=False, alpha=0.05)

* Quantifies uncertanty around an average value over many objects.

#### Prediction Interval

In [ ]:
prediction.conf_int(obs=True, alpha=0.05)

* Quantifies uncertanty around a value for a particular object.

## Analyze the model

### Collinearity

#### Threats
- Uncertain parameters.
- Slight change of the data will significantly change the model.

#### How to find?
- Variance Inflation Factor: ratio of $\sigma_{\hat{\beta}}$ in full model to $\sigma_{\hat{\beta}}$ in one-feature model.
- Correlation Matrix.

#### How to fix?
- Drop features.
- PCA.

In [ ]:
def measure_vif(df: pd.DataFrame):
    vifs = [VIF(X, i) for i in range(df.shape[1])]
    vifs = pd.DataFrame({'vif':vifs}, index=X.columns)
    display(vifs[(vifs > 5).values.flatten()])

def color_positive_green(val):
    white = np.array([255, 255, 255])
    if val >= 0:
        green = np.array([0, 255, 0])
        color = green * val + white * (1 - val)
    else:
        val = -val
        red = np.array([255, 0, 0])
        color = red * val + white * (1 - val)
    r, g, b = color.round()
    
    return 'background-color: %s' % f"rgb({r},{g},{b})"

def show_corr(df:pd.DataFrame, min_corr: float = 0.7):
    tmp = df.corr(method='spearman')
    tmp = tmp[tmp.abs() > min_corr].style.applymap(color_positive_green)
    display(tmp)

In [ ]:
features = ['crim', 'zn', 'indus', 'chas', 'nox', 'rm', 'age', 'dis', 'rad', 'tax', 'ptratio', 'lstat', 'medv']
X = Boston[features].copy()
X['intercept'] = 1.0
measure_vif(X)
show_corr(X, min_corr=0.7)

### Outliers

#### How to find?
* Points with big residuals.
* Can have little impact on the fitted line.
* Spoil metrics and estimates: CI, $p$-value, $R^2$, RSE.

#### How to fix?

* Studentize and exclude > 3.

In [ ]:
tmp = results.outlier_test()
X[tmp['student_resid'].abs() > 3]

### High leverage points

#### How to find?
* Points with unusual values of features (either out of common domain or unusual combination).
* Simple linear regression:
    $$h_i = \frac{1}{n} + \frac{(x_i - \bar{x})^2}{\sum_j (x_j - \bar{x})^2}$$
* Multiple regression: diagonal elements of hat matrix $H = X (X^T X)^{-1} X^T$

In [ ]:
infl = results.get_influence()

plt.scatter(np.arange(X.shape[0]), infl.hat_matrix_diag)
plt.xlabel('Index')
plt.ylabel('Leverage')

n = X.shape[0]
p = len(features) + 1
threshold = 3 * p / n
plt.axhline(y = threshold, color = 'r', linestyle = '-')

mask = infl.hat_matrix_diag > threshold
X[mask]

#### How to fix?
* Consider and exclude if they are either non-typical or erroneous.

### Non-linearity

#### How to find?
* We analyze residuals in axis $\hat{y} - y$ vs $\hat{y}$.
* Ideally, there should be no dependence between fitted values and residuals.
* Any discernable pattern indicates a problem with linearity.

In [ ]:
plt.scatter(results.fittedvalues, results.resid)
plt.xlabel('Fitted value')
plt.ylabel('Residual')
plt.axhline(0, c='k', ls='--');

#### How to fix?

* Apply non-linear transformations to the features.